In [8]:
# script to identify top parameters (according to local uncertainty) to perturb for global uncertainty

In [ ]:
import os
import re
import sys
import glob
import copy
import yaml
import pickle
import subprocess
import numpy as np
import pandas as pd

import rmgpy.data.kinetics
import rmgpy.chemkin
import cantera as ct


sys.path.append(os.path.join(os.environ['AUTOSCIENCE_REPO'], 'database'))
import database_fun

import matplotlib.pyplot as plt
%matplotlib inline




In [2]:
def get_reaction_indices(reaction_list, ref_rxn):
    reaction_indices = []
    for i in range(len(reaction_list)):
        if reaction_list[i].is_isomorphic(ref_rxn):
            reaction_indices.append(i)
    return reaction_indices

In [3]:
# get the top N sensitive parameters by reading in the first-derivative
N = 10

analysis_dir = '/work/westgroup/harris.se/autoscience/fuels/butane/official/butane_20240126/analysis'
    
# load the RMG mech 
chemkin = os.path.join(analysis_dir, '..', 'chem_annotated.inp')
species_dict = os.path.join(analysis_dir, '..', 'species_dictionary.txt')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(chemkin, species_dict)
with open(os.path.join(analysis_dir, '..', 'ct2rmg_rxn.pickle'), 'rb') as f:
    ct2rmg_rxn = pickle.load(f)


# load from files
total_uncertainty_mat = np.load(os.path.join(analysis_dir, 'total_uncertainty_mat.npy'))
dft_uncertainty_mat = np.load(os.path.join(analysis_dir, 'dft_uncertainty_mat.npy'))
first_derivative = np.load(os.path.join(analysis_dir, 'first_derivative.npy'))


Tmax = 1077  # use min and max temperature range of the data: 663K-1077K
Tmin = 663
K = 51
table_index = 7

my_sensitivities = first_derivative[:, (table_index - 1) * K: table_index * K][:, 26]
my_sensitivities[np.isnan(my_sensitivities)] = 0
my_uncertainties = total_uncertainty_mat[:, (table_index - 1) * K: table_index * K][:, 26]
my_uncertainties[np.isnan(my_uncertainties)] = 0
my_contribution = np.multiply(my_uncertainties, np.abs(my_sensitivities))

# get the order
abs_parameter_indices = np.arange(0, my_sensitivities.shape[0])
abs_contribution_order = [x for _, x in sorted(zip(my_contribution, abs_parameter_indices))][::-1]

count = 0
top_species = []
top_reactions = []

for j in range(N):
    ct_index = abs_contribution_order[j]
    if ct_index < len(species_list):
        top_species.append(ct_index)
        print(j, 'species', species_list[ct_index])
        continue
    else:
        top_reactions.append(ct2rmg_rxn[ct_index - len(species_list)])
        db_index = database_fun.get_unique_reaction_index(reaction_list[ct2rmg_rxn[ct_index - len(species_list)]])
        print(j, db_index, str(reaction_list[ct2rmg_rxn[ct_index - len(species_list)]]))


0 species O2(2)
1 288 OH(15) + butane(1) <=> H2O(8) + [CH2]CCC(24)
2 species butane(1)
3 4778 C[CH]CC(23) + CCCCOO(225) <=> CCCCO[O](215) + butane(1)
4 313 O2(2) + [CH2]CCC(24) <=> HO2(16) + C=CCC(47)
5 species C[CH]CC(23)
6 5046 COO(94) + C[CH]CC(23) <=> CO[O](30) + butane(1)
7 species CC(CCOO)O[O](181)
8 species HO2(16)
9 4752 CC(CCOO)O[O](181) <=> OH(15) + CC(CC=O)OO(177)


In [ ]:
# These are the values you will manually enter for the run_all_sims.py script

In [6]:
top_species

[5, 4, 24, 62, 19]

In [7]:
top_reactions

[84, 238, 163, 689, 142]